In [1]:
import numpy as np
import pandas as pd
import xgboost # xgboost

from xgboost import XGBRegressor # XGBRegressor 호출
from sklearn.model_selection import train_test_split # 훈련데이터 나누는 함수
from sklearn.model_selection import GridSearchCV # 모델 파라미터 최적치 찾아주는 함수
from sklearn.metrics import accuracy_score # 정확도
from sklearn.metrics import explained_variance_score # 그거 수업때 1에 가까우면 좋은 모델인 수치

from sklearn.model_selection import KFold # 교차검증 >> 시도중 >> 근데 하면 훨씬 점수가 올라가겠지만 그만큼 시간이... 수배로...
                                            # 찾다보니 xgboost에 이미 교차검증이 들어있다는데 그것이 파라미터를 설정해야 동작하는
                                            # 것인지는 모르겠음 

C:\Users\AI\Anaconda3\lib\site-packages\xgboost\compat.py:31: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
train = pd.read_csv("./train.csv")
test = pd.read_csv("./test.csv") # 데이터 불러오기

In [3]:
total_data = pd.concat([train, test])

total_data = pd.get_dummies(total_data)

train_data = total_data[~total_data['PRICE'].isnull()]

test_data = total_data[total_data['PRICE'].isnull()]

test_data = test_data.drop('PRICE', axis=1) # 두개 병합하고 인코딩, 테스트는 가격컬럼 없었으니 드랍

In [4]:
X = train_data.iloc[:,2:]
y = train_data.iloc[:,1] # 모델링 데이터 형식 만들기 : 앞에 컬럼 하나 버렸음

In [5]:
X_train_full, X_test, y_train_full, y_test=train_test_split(X,y,random_state=777) # 나누는 함수

In [6]:
X_train, X_val, y_train, y_val=train_test_split(X_train_full, y_train_full, random_state=777) # 한번 더 나눔

In [1]:
my_model =  (n_estimators = 600, max_depth = 6,min_child_weight= 1.2, colsample_bytree = 1.0, n_jobs = -1)

my_model.fit(X_train, y_train)

# reg_cv = GridSearchCV(my_model, {"colsample_bytree":[1.0],"min_child_weight":[1.0,1.2],'max_depth': [3,4,6,7,8,9,10], 'n_estimators': [500,1000,1500,2000,2500]}, verbose=1)
# reg_cv.fit(X_train, y_train)

# reg_cv.best_params_

# 기본 세부 파라미터 지정, 주석 처리된 GridSearchCV로 최적 파라미터 찾을 수 있음 (※안의 수치 조정안하면 상상 이상의 시간이 날아감)
# 모든 세부 파라미터 설명은 라이브러리 사이트에서 원문으로 찾아보는 것이 가장 정확함
# 저 수치들이 전부가 아닐 수 있으나 인터넷에서 긁어왔음 + 저것도 다 고려할 수 없음

SyntaxError: invalid syntax (2190384774.py, line 1)

In [8]:
best_model = my_model

# best_model = reg_cv.best_estimator_

# 베스트 모델에 파라미터 지정한 내 모델 넣거나 // 주석된 것은 GridSearchCV로 찾은 최적값 파라미터가 넣어진 모델을 넣음 

from xgboost import XGBRegressor

my_model = XGBRegressor(n_estimators = 500, max_depth = 9,min_child_weight= 1.2,colsample_bytree = 1.0,n_jobs = -1)
my_model.fit(train_X, train_y, verbose=False)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100,
             n_jobs=-1, num_parallel_tree=1, predictor='auto', random_state=0,
             reg_alpha=0, reg_lambda=1, ...)

In [9]:
pred_test = best_model.predict(X_test)
r_sq = best_model.score(X_train, y_train)
print(r_sq)
print(explained_variance_score(pred_test,y_test))

# r_sq가 위에서 나눈 test 예측값 점수, 밑 수치가 방금 수업에서 말한 1에 가까우면 좋은 모델인 수치

0.9999999977320858
0.7092345645558062


In [10]:
pred_train = best_model.predict(X_train_full)
r_sq2 = best_model.score(X_train_full, y_train_full)
print(r_sq2)
print(explained_variance_score(pred_train,y_train_full))

# r_sq가 위에서 나눈 train 예측값 점수, 밑 수치가 방금 수업에서 말한 1에 가까우면 좋은 모델인 수치

0.9505059035187825
0.9485183192162443


In [11]:
prediction1 = best_model.predict(test_data.iloc[:,1:])

# 이게 전처리한 TEST 데이터 예측, 답을 모르니 정확도는 측정불가

In [12]:
prediction1

array([ 554656.94, 1487638.5 , 1000287.2 , ..., 1027779.6 ,  621832.56,
        368954.78], dtype=float32)

In [13]:
print(prediction1.sum()/15146) 

# 대강 평균 가격 비교

638828.0723623399


In [14]:
pred_test.sum()/len(pred_test)

# 대강 평균 가격 비교

636642.1296456353

In [15]:
prediction1 = pd.DataFrame(prediction1)
prediction1.to_csv("./xgboost_opt5.csv")

# 뒤에는 귀찮아서 복붙 수정해서 규격 맞춰서 올림